In [38]:
using Optimization, OptimizationOptimJL, Plots
using ModelingToolkit, IntervalSets
using Sophon
using ChainRulesCore

@parameters t, x
@variables u(..), ρ(..), p(..)
Dₓ = Differential(x)
Dₜ = Differential(t)

u₀(x) = 0.0
ρ₀(x) = ifelse(x < 0.5, 1.0, 0.125)
@register_symbolic ρ₀(x)
p₀(x) = ifelse(x < 0.5, 1.0, 0.1)
@register_symbolic p₀(x)

bcs = [ρ(0, x) ~ ρ₀(x), u(0, x) ~ u₀(x), p(0, x) ~ p₀(x), u(t, 0) ~ 0.0, u(t, 1) ~ 0.0]

γ = 1.4
E(t, x) = p(t, x) / (γ - 1) + 0.5 * ρ(t, x) * abs2(u(t, x))

eqs = [
    Dₜ(ρ(t, x)) + Dₓ(ρ(t, x) * u(t, x)) ~ 0.0,
    Dₜ(ρ(t, x) * u(t, x)) + Dₓ(ρ(t, x) * u(t, x) * u(t, x) + p(t, x)) ~ 0.0,
    Dₜ(E(t, x)) + Dₓ(u(t, x) * (E(t, x) + p(t, x))) ~ 0.0,
]

t_min, t_max = 0.0, 0.2
x_min, x_max = 0.0, 1.0
domains = [t ∈ Interval(t_min, t_max), x ∈ Interval(x_min, x_max)]

@named pde_system = PDESystem(eqs, bcs, domains, [t, x], [u(t, x), ρ(t, x), p(t, x)])

PDESystem
Equations: Equation[Differential(x)(u(t, x)*ρ(t, x)) + Differential(t)(ρ(t, x)) ~ 0.0, Differential(x)((u(t, x)^2)*ρ(t, x) + p(t, x)) + Differential(t)(u(t, x)*ρ(t, x)) ~ 0.0, Differential(t)(2.5000000000000004p(t, x) + 0.5abs2(u(t, x))*ρ(t, x)) + Differential(x)((3.5000000000000004p(t, x) + 0.5abs2(u(t, x))*ρ(t, x))*u(t, x)) ~ 0.0]
Boundary Conditions: Equation[ρ(0, x) ~ ρ₀(x), u(0, x) ~ 0.0, p(0, x) ~ p₀(x), u(t, 0) ~ 0.0, u(t, 1) ~ 0.0]
Domain: Symbolics.VarDomainPairing[Symbolics.VarDomainPairing(t, 0.0..0.2), Symbolics.VarDomainPairing(x, 0.0..1.0)]
Dependent Variables: Num[u(t, x), ρ(t, x), p(t, x)]
Independent Variables: Num[t, x]
Parameters: SciMLBase.NullParameters()
Default Parameter ValuesDict{Any, Any}()

In [39]:
pinn = PINN(u=FullyConnected(2, 1, tanh; num_layers=4, hidden_dims=16),
            ρ=FullyConnected(2, 1, tanh; num_layers=4, hidden_dims=16),
            p=FullyConnected(2, 1, tanh; num_layers=4, hidden_dims=16))

sampler = QuasiRandomSampler(1000, 400)

function pde_weights(phi, x, θ)
    ux = Sophon.finitediff(phi.u, x, θ.u, 1, 1)
    ρx = Sophon.finitediff(phi.ρ, x, θ.ρ, 1, 1)
    px = Sophon.finitediff(phi.p, x, θ.p, 1, 1)
    d = ux .+ ρx .+ px

    return ChainRulesCore.@ignore_derivatives inv.(0.2 .* abs.(d) .+ 1)
end

strategy = AdaptiveTraining(pde_weights, Returns(10))
prob = Sophon.discretize(pde_system, pinn, sampler, strategy)

OptimizationProblem. In-place: true
u0: ComponentVector{Float64}(u = (layer_1 = (weight = [0.2695924937725067 1.9268028736114502; -0.21252557635307312 -0.7415732145309448; … ; 0.7171503901481628 -1.5442533493041992; -0.9443339705467224 -1.5292670726776123], bias = [0.0; 0.0; … ; 0.0; 0.0;;]), layer_2 = (weight = [-0.6643564701080322 -0.6925046443939209 … 0.008272568695247173 -0.21165762841701508; -0.16472826898097992 0.6727578639984131 … 0.312429279088974 0.3575981855392456; … ; -0.40722420811653137 0.5163592100143433 … 0.5410106778144836 0.501392126083374; 0.4628823697566986 -0.28122982382774353 … -0.09267458319664001 0.5269978642463684], bias = [0.0; 0.0; … ; 0.0; 0.0;;]), layer_3 = (weight = [0.07574213296175003 0.0266563780605793 … 0.13516564667224884 0.11330641061067581; 0.09291994571685791 -0.1699279546737671 … 0.495624303817749 0.6320878863334656; … ; -0.5844491124153137 -0.6654408574104309 … -0.49571728706359863 0.4476800858974457; -0.4677067697048187 -0.5325641632080078 … 0.20

In [40]:
res = Optimization.solve(prob, BFGS(); maxiters=1000)
res.objective

In [ ]:
using Plots
θ = res.u
phi = pinn.phi
xs = x_min:0.01:x_max

phi = pinn.phi
p1 = plot(xs, [first(phi.u([t_max, x], θ.u)) for x in xs]; label="u(t=1,x)")
p2 = plot!(xs, [first(phi.ρ([t_max, x], θ.ρ)) for x in xs]; label="ρ(t=1,x)")
p3 = plot!(xs, [first(phi.p([t_max, x], θ.p)) for x in xs]; label="p(t=1,x)")